# Example 3: A Buoyant Bubble

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-research/swirl-lm/blob/main/swirl_lm/example/swirl_lm_public_demo3_buoyant_bubble.ipynb)


The example in this colab shows how to run a buoyant bubble simulation with Swirl-LM on TPU. It shows how libraries in Swirl-LM is loaded and build, and the key components to set up a simulation with Swirl-LM.

Note that this colab requires connection to a runtime with TPU. Before you run this Colab notebook, make sure that your hardware accelerator is a TPU by checking your notebook settings: **Runtime** > **Change runtime type** > **Hardware accelerator** > **TPU**. The default TPU runtime has 8 cores available.

In [ ]:
!git clone https://github.com/google-research/swirl-lm.git
!./swirl-lm/swirl_lm/setup.sh
!python3 -m pip show swirl-lm


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from swirl_lm.base import driver
from swirl_lm.base import driver_tpu
from swirl_lm.base import initializer
from swirl_lm.base import parameters
from swirl_lm.physics.thermodynamics import thermodynamics_manager
from swirl_lm.utility import get_kernel_fn
from swirl_lm.utility import tpu_util


# Simulation Setup
To initialize a simulation, one needs to provide the following:
   * A text protobuf that specifies the simulation configuration (e.g. domain
     size, partitions) and user defined parameters for a simulation
   * A text protobuf file that specifies the solver parameters and physical
     conditions of a simulation, e.g. boundary conditions
   * A function that initializes state variables to start the simulation

In [ ]:
# Simulation configuration.
params = parameters.SwirlLMParameters.config_from_text_proto(
    """
  # proto-file: swirl_lm/base/parameters.proto
  # proto-message: SwirlLMParameters
  solver_procedure: VARIABLE_DENSITY
  convection_scheme: CONVECTION_SCHEME_QUICK
  diffusion_scheme: DIFFUSION_SCHEME_STENCIL_3
  time_integration_scheme: TIME_SCHEME_CN_EXPLICIT_ITERATION
  num_sub_iterations: 3
  enable_rhie_chow_correction: false
  grid_params {
    # The number of cores in 3 dimensions.
      computation_shape {
        dim_0: 4
        dim_1: 2
        dim_2: 1
      }
    # The physical size of the simulation domain in units of m.
    length {
      dim_0: 2e4
      dim_1: 1e4
      dim_2: 40.0
    }
    # The number of grid points per core in 3 dimensions including ghost cells
    # (halos).
    grid_size {
      dim_0: 128
      dim_1: 128
      dim_2: 6
    }
    # The width of the ghost cells on each side of the domain. It is set to 2
    # considering the stencil width of the QUICK scheme.
    halo_width: 2
    # The time step size in units of s.
    dt: 0.3
    # The size of the convolution kernel to be used for fundamental numerical
    # operations.
    kernel_size: 16
    periodic {
      dim_0: false
      dim_1: false
      dim_2: true
    }
  }
  gravity_direction {
    dim_0: 0.0
    dim_1: -1.0
    dim_2: 0.0
  }
  pressure {
    solver {
      jacobi {
        max_iterations: 10 halo_width: 2 omega: 0.67
      }
    }
    num_d_rho_filter: 3
  }
  p_thermal: 1.01325e5
  kinematic_viscosity: 25.0
  use_sgs: true
  sgs_model {
    smagorinsky {
      c_s: 0.18
      pr_t: 1.0
      use_pr_t: false
    }
  }
  thermodynamics {
    solver_mode: LOW_MACH
    ideal_gas_law {
      const_theta: 300.0
      cv_d: 716.9
    }
  }
  scalars {
    name: "theta"
    diffusivity: 25.0
    solve_scalar: true
  }
  scalars {
    name: "ambient"
    diffusivity: 25.0
    density: 1.22
    molecular_weight: 0.02875
    solve_scalar: false
  }
  additional_state_keys: "zz"
  additional_state_keys: "nu_t"
  boundary_conditions {
    name: "u"
    boundary_info {
      dim: 0 location: 0 type: BC_TYPE_DIRICHLET value: 0.0
    }
    boundary_info {
      dim: 0 location: 1 type: BC_TYPE_DIRICHLET value: 0.0
    }
    boundary_info {
      dim: 1 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
  }
  boundary_conditions {
    name: "v"
    boundary_info {
      dim: 0 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 0 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 0 type: BC_TYPE_DIRICHLET value: 0.0
    }
    boundary_info {
      dim: 1 location: 1 type: BC_TYPE_DIRICHLET value: 0.0
    }
  }
  boundary_conditions {
    name: "w"
    boundary_info {
      dim: 0 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 0 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
  }
  boundary_conditions {
    name: "p"
    boundary_info {
      dim: 0 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 0 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
  }
  boundary_conditions {
    name: "theta"
    boundary_info {
      dim: 0 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 0 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 0 type: BC_TYPE_NEUMANN value: 0.0
    }
    boundary_info {
      dim: 1 location: 1 type: BC_TYPE_NEUMANN value: 0.0
    }
  }
  """,
    grid_params_from_flags=False,
)

In [ ]:
# Defines the function that initializes state variables.
_T_BASE = 300.0
_T_PERT = 2.0
_X_C = 0.5 * params.lx
_Y_C = 2e3
_X_R = 2e3
_Y_R = 2e3


def _initialize_states(coordinates, value_fn):
  """Generates partial states for core using `value_fn`."""
  return initializer.partial_mesh_for_core(
      params,
      coordinates,
      value_fn,
      pad_mode='SYMMETRIC',
      mesh_choice=initializer.MeshChoice.PARAMS,
  )


def _init_fn_potential_temperature(xx, yy, zz, lx, ly, lz, coord):
  """Initializes the potential temperature."""
  del zz, lx, ly, lz, coord

  r = tf.math.sqrt(((xx - _X_C) / _X_R) ** 2 + ((yy - _Y_C) / _Y_R) ** 2)

  return _T_BASE + tf.where(
      r < 1.0, _T_PERT * tf.math.cos(0.5 * np.pi * r) ** 2, tf.zeros_like(r)
  )


def _init_fn_constant(c):
  """Generates a function that initializes the flow field to a constant `c`."""

  def init_fn(xx, yy, zz, lx, ly, lz, coord):
    """Initializes the flow field to `c`."""
    del xx, zz, lx, ly, lz, coord

    return c * tf.ones_like(yy)

  return init_fn


def init_fn_buoyant_bubble(replica_id, coordinates):
  """Initializes state variables in a buoyant bubble simulation."""
  init_fn_zz = lambda xx, yy, zz, lx, ly, lz, coord: yy

  output = {
      'replica_id': replica_id,
      'u': _initialize_states(coordinates, _init_fn_constant(0.0)),
      'v': _initialize_states(coordinates, _init_fn_constant(0.0)),
      'w': _initialize_states(coordinates, _init_fn_constant(0.0)),
      'p': _initialize_states(coordinates, _init_fn_constant(0.0)),
      'theta': _initialize_states(coordinates, _init_fn_potential_temperature),
      'zz': _initialize_states(coordinates, init_fn_zz),
      'nu_t': _initialize_states(coordinates, _init_fn_constant(0.0)),
  }

  # Initialize the density that is consistent with the temperature field.
  thermodynamics_model = thermodynamics_manager.thermodynamics_factory(
      params
  ).model
  thermal_states = {'theta': [output['theta']]}
  additional_states = {'zz': [output['zz']]}
  output.update({
      'rho': thermodynamics_model.update_density(
          thermal_states, additional_states
      )[0]
  })

  return output

# TPU initialization

In [ ]:
# Initializes the TPU strategy.
computation_shape = np.array([params.cx, params.cy, params.cz])

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=''
  )  # TPU detection
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
topology = tf.tpu.experimental.initialize_tpu_system(tpu)
device_assignment, _ = tpu_util.tpu_device_assignment(
    computation_shape=computation_shape, tpu_topology=topology
)
tpu_strategy = tf.distribute.experimental.TPUStrategy(
    tpu, device_assignment=device_assignment
)
logical_coordinates = tpu_util.grid_coordinates(computation_shape).tolist()

print('All devices: ', tf.config.list_logical_devices('TPU'))

# Run the simulation

In [ ]:
# initializes the simulation.
state = driver_tpu.distribute_values(
    tpu_strategy,
    value_fn=init_fn_buoyant_bubble,
    logical_coordinates=logical_coordinates,
)

In [ ]:
%%time
# The number of time steps to run before restart data files are written. The
# completion of these steps is considered as a cycle.
num_steps = 4000

# Runs the simulation for one cycle.
step_id = tf.constant(0)
kernel_op = get_kernel_fn.ApplyKernelConvOp(params.kernel_size)
model = driver._get_model(kernel_op, params)

state, completed_steps, _, _ = driver._one_cycle(
    strategy=tpu_strategy,
    init_state=state,
    init_step_id=step_id,
    num_steps=num_steps,
    params=params,
    model=model)

Note that the runtime for the code block above is long. This is due to the JIT compilzation of the TensorFlow graph when a function is called for the first time. The compiled graph will be used for subsequent calls to the same function with the same input type. The actual runtime for 100 steps in this example is
around 10 ms (try running the code block below and see how the timing changes).

In [ ]:
%%time
step_id += num_steps

state, completed_steps, _, _ = driver._one_cycle(
    strategy=tpu_strategy,
    init_state=state,
    init_step_id=step_id,
    num_steps=num_steps,
    params=params,
    model=model)

# Post-processing

In [ ]:
# Utility functions for postprocessing.


def merge_result(values, coordinates, halo_width):
  """Merges results from multiple TPU replicas following the topology."""
  if len(values) != len(coordinates):
    raise (
        ValueError,
        (
            'The length of `value` and `coordinates` must equal. Now `values`'
            f' has length {len(values)}, but `coordinates` has length'
            f' {len(coordinates)}.'
        ),
    )

  # The results are oriented in order z-x-y.
  nz, nx, ny = values[0].shape
  nz_0, nx_0, ny_0 = [n - 2 * halo_width for n in (nz, nx, ny)]

  # The topology is oriented in order x-y-z.
  cx, cy, cz = np.array(np.max(coordinates, axis=0)) + 1

  # Compute the total size without ghost cells/halos.
  shape = [n * c for n, c in zip([nz_0, nx_0, ny_0], [cz, cx, cy])]

  result = np.empty(shape, dtype=np.float32)

  for replica in range(len(coordinates)):
    s = np.roll(
        [c * n for c, n in zip(coordinates[replica], (nx_0, ny_0, nz_0))],
        shift=1,
    )
    e = [s_i + n for s_i, n in zip(s, (nz_0, nx_0, ny_0))]
    result[s[0] : e[0], s[1] : e[1], s[2] : e[2]] = values[replica][
        halo_width : nz_0 + halo_width,
        halo_width : nx_0 + halo_width,
        halo_width : ny_0 + halo_width,
    ]

  return result

In [ ]:
# @title Results visualization
varname = 'theta'  # @param ['u', 'v', 'w', 'p', 'rho', 'theta', 'zz', 'nu_t']

if np.prod(computation_shape) == 1:
  result = state[varname].numpy()[
      params.halo_width : -params.halo_width,
      params.halo_width : -params.halo_width,
      params.halo_width : -params.halo_width,
  ]
else:
  result = merge_result(
      state[varname].values, logical_coordinates, params.halo_width
  )

nx = (params.nx - 2 * params.halo_width) * params.cx
ny = (params.ny - 2 * params.halo_width) * params.cy
nz = (params.nz - 2 * params.halo_width) * params.cz

x = np.linspace(0.0, params.lx, nx)
y = np.linspace(0.0, params.ly, ny)
z = np.linspace(0.0, params.lz, nz)

fig, ax = plt.subplots(figsize=(18, 6))
c = ax.contourf(x, y, result[nz // 2, ...].transpose(), cmap='jet', levels=21)
fig.colorbar(c)
ax.axis('equal')
plt.show()